# Introduction
This notebook is a re-implementation of Fast AI's lesson 10 notebook.

The goal is to implement some basic matrix multiplication operations.

We are allowed to use the following libraries:
- Python and the included standard library
- matplotlib
- Jupyter notebooks and nbdev

In [2]:
from pathlib import Path
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, matplotlib.pyplot as plt

# Get data
This section is copied from the existing notebook.

In [3]:
MNIST_URL = 'https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [4]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [5]:
!ls -l data

total 33312
-rw-r--r--  1 pj  staff  17051982 May 24 19:17 mnist.pkl.gz


In [6]:
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [7]:
x_train[0].shape

(784,)

In [8]:
x_train[0][20:30]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [9]:
img0 = x_train[0]

# Converting the 1D list to a 2D matrix
1. Using `yield`
2. Using `iter`/`islice`

## `yield`

In [10]:
def chunks(input_list, row_length):
  for i in range(0, len(input_list), row_length): yield input_list[i: i+row_length]

In [11]:
img0_yield_2d = list(chunks(img0, 28))

In [12]:
len(img0_yield_2d)

28

In [13]:
len(img0_yield_2d[0])

28

## `iter`/`islice`

In [14]:
from itertools import islice

In [15]:
iterator = iter(img0)
row_length = 28

img0_islice_2d = [list(islice(iterator, row_length)) for _ in range(len(img0) // row_length)]

In [16]:
len(img0_islice_2d)

28

In [17]:
len(img0_islice_2d[0])

28

## Compare `yield` and `islice`'s speed

In [18]:
%timeit -n 10 list(chunks(img0, 28))

4.53 µs ± 824 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit -n 10 [list(islice(iterator, row_length)) for _ in range(len(img0) // row_length)]

4.38 µs ± 539 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


I find it interesting that `chunks` works slightly faster, but its standard deviation is much larger than that of `islice`.

# Implementing matrix

In [20]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix

    def __getitem__(self, idxs):
        return self.matrix[idxs[0]][idxs[1]]

In [21]:
img0_mat = Matrix(img0_islice_2d)

In [22]:
img0_mat[2,15]

0.0

In [23]:
print(img0_islice_2d[2][15])

0.0


Ok! Now we have implemented basic matrix operations in Python, we can use Pytorch.

In [25]:
import torch

# Matrix multiplication from scratch

In [33]:
m1 = x_train[:5]; m1.shape
m2 = torch.randn(784, 10)

## Naive nested loop for matrix multiplication

In [34]:
def matmul(m1, m2):
    (m1r, m1c), (m2r, m2c) = m1.shape, m2.shape
    res = torch.zeros(m1r, m2c)
    for i in range(m1r):
        for j in range(m1c):
            for k in range(m2c):
                res[i, k] += m1[i, j] * m2[j, k]

    return res

In [35]:
m3 = matmul(m1, m2)

In [37]:
m3

tensor([[ -4.4936,  -5.6175, -22.5178,  -3.9274, -12.2499,   3.0905,  -7.8525,
          -0.4918,  -1.8969, -15.1004],
        [ -0.5469,   2.9690, -17.8994,  -6.0521, -27.5086,   9.8671,  -9.4390,
          -1.5734,  -8.8437,  -7.7993],
        [ -4.8420,   7.3685,  -7.9591, -11.0315,   6.6914, -10.7308,  12.4244,
           8.7850,   8.4482,  -2.8377],
        [ -4.1739,   1.5831, -10.6296,  -1.7576,  -1.8362,  12.7535,  -9.5721,
          14.2302,  -5.5941, -10.1399],
        [ -1.0859, -12.7273,  -0.1452,  -6.9904,  -9.4698,  19.9961,   3.7399,
          -2.0317,  -2.5721,  -9.6477]])

In [42]:
%timeit matmul(m1, m2)

411 ms ± 7.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numba for dot product function

In [51]:
from numba import jit
import numpy as np

@jit(nopython=True)
def dot(a, b):
    summation = 0
    for i in range(a.shape[0]):
        summation += a[i] * b[i]

    return summation
    

In [70]:
def matmul(m1, m2):
    (m1r, m1c), (m2r, m2c) = m1.shape, m2.shape
    res = torch.zeros(m1r, m2c)
    for i in range(m1r):
        for j in range(m2c):
                a, b = m1[i,:], m2[:, j]
                res[i, j] = dot(a, b)

    return res

In [67]:
# convert m2 from torch tensor to numpy as numba needs np
m2 = m2.numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [71]:
matmul(m1, m2)

tensor([[ -4.4936,  -5.6175, -22.5178,  -3.9274, -12.2499,   3.0905,  -7.8525,
          -0.4918,  -1.8969, -15.1004],
        [ -0.5469,   2.9690, -17.8994,  -6.0521, -27.5086,   9.8671,  -9.4390,
          -1.5734,  -8.8437,  -7.7993],
        [ -4.8420,   7.3685,  -7.9591, -11.0315,   6.6914, -10.7308,  12.4244,
           8.7850,   8.4482,  -2.8377],
        [ -4.1739,   1.5831, -10.6296,  -1.7576,  -1.8362,  12.7535,  -9.5721,
          14.2302,  -5.5941, -10.1399],
        [ -1.0859, -12.7273,  -0.1452,  -6.9904,  -9.4698,  19.9961,   3.7399,
          -2.0317,  -2.5721,  -9.6477]])

In [72]:
%timeit matmul(m1, m2)

208 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Pytorch Implementation

In [73]:
def matmul(m1, m2):
    (m1r, m1c), (m2r, m2c) = m1.shape, m2.shape
    res = torch.zeros(m1r, m2c)
    for i in range(m1r):
        for j in range(m2c):
                a, b = m1[i,:], m2[:, j]
                res[i, j] = torch.sum(a * b)

    return res

In [76]:
m1 = torch.from_numpy(m1)
m2 = torch.from_numpy(m2)

matmul(m1, m2)

tensor([[ -4.4936,  -5.6175, -22.5178,  -3.9274, -12.2499,   3.0905,  -7.8525,
          -0.4918,  -1.8969, -15.1004],
        [ -0.5469,   2.9690, -17.8994,  -6.0521, -27.5086,   9.8671,  -9.4390,
          -1.5734,  -8.8437,  -7.7993],
        [ -4.8420,   7.3685,  -7.9591, -11.0315,   6.6914, -10.7308,  12.4244,
           8.7850,   8.4482,  -2.8377],
        [ -4.1739,   1.5831, -10.6296,  -1.7576,  -1.8362,  12.7535,  -9.5721,
          14.2302,  -5.5941, -10.1399],
        [ -1.0859, -12.7273,  -0.1452,  -6.9904,  -9.4698,  19.9961,   3.7399,
          -2.0317,  -2.5721,  -9.6477]])

In [77]:
%timeit matmul(m1, m2)

479 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Broadcasting
The examples below illustrate how we can use broadcasting to help with our matrix multiplication.

In [79]:
from torch import tensor

c = tensor([10.,20,30]); c

tensor([10., 20., 30.])

In [80]:
c.shape

torch.Size([3])

We can put `None` at an axis that does not exist to add a dimension at that axis. Works the same way as unsqueeze.

In [81]:
c[None,:]

tensor([[10., 20., 30.]])

In [82]:
c[None,:].shape

torch.Size([1, 3])

We see that an extra dimension has indeed been added. We went from a vector to a matrix.

In [83]:
c[:, None]

tensor([[10.],
        [20.],
        [30.]])

In [84]:
c[:, None].shape

torch.Size([3, 1])

Because we want to add an extra dimension at the second dimension (column), we convert our vector of 3 elements to 3 rows of 1 element each.

In [85]:
c[None, :] * c[:, None]

tensor([[100., 200., 300.],
        [200., 400., 600.],
        [300., 600., 900.]])

We have 2 matrices of 1 by 3 and 3 by 1. In order to accomodate the element wise multiplication, our first element `c[None, :]` of 1 by 3 has its first dimension 'expanded' (in reality we use a stride of 0) to become a 3 by 3 matrix. In other words, the first dimension of 1 is expanded to 3 to match the second element which has 3 rows.

A similar logic applies for `c[:, None]`, where it has its second dimension expanded from 1 to 3 in order to match the second dimension of `c[None, :]`.

Thus, we end up with 2 matrices with dimensions 3 by 3.

In [86]:
c[None] > c[:,None]

tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

A similar logic applies for boolean operation too.

In [87]:
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]]); m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [88]:
m*m

tensor([[ 1.,  4.,  9.],
        [16., 25., 36.],
        [49., 64., 81.]])



When operating on two arrays/tensors, Numpy/PyTorch compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when
- they are equal, or
- one of them is 1, in which case that dimension is broadcasted to make it the same size

Arrays do not need to have the same number of dimensions. For example, if you have a 256*256*3 array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3

The numpy documentation includes several examples of what dimensions can and can not be broadcast together.


### Matmul with broadcasting

In [103]:
def matmul(m1, m2):
    (m1r, m1c), (m2r, m2c) = m1.shape, m2.shape
    res = torch.zeros(m1r, m2c)
    for i in range(m1r):
      digit = m1[i]
      res[i] = (digit[:,None] * m2).sum(axis=0)

    return res

In [104]:
matmul(m1, m2)

tensor([[ -4.4936,  -5.6175, -22.5178,  -3.9274, -12.2499,   3.0905,  -7.8525,
          -0.4918,  -1.8969, -15.1004],
        [ -0.5469,   2.9690, -17.8994,  -6.0521, -27.5086,   9.8671,  -9.4390,
          -1.5734,  -8.8437,  -7.7993],
        [ -4.8420,   7.3685,  -7.9591, -11.0315,   6.6914, -10.7308,  12.4244,
           8.7850,   8.4482,  -2.8377],
        [ -4.1739,   1.5831, -10.6296,  -1.7576,  -1.8362,  12.7535,  -9.5721,
          14.2302,  -5.5941, -10.1399],
        [ -1.0859, -12.7273,  -0.1452,  -6.9904,  -9.4698,  19.9961,   3.7399,
          -2.0317,  -2.5721,  -9.6477]])

In [105]:
%timeit matmul(m1, m2)

57.6 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


We have obtained a speed-up of 7000x!